# Robot Sensors

Kevin J. Walchko, 24 Sept 2017

---

Blah ...

## References

- [Dead Reckoning Wikipedia](https://en.wikipedia.org/wiki/Dead_reckoning)
- [Interrupts](http://raspi.tv/2013/how-to-use-interrupts-with-python-on-the-raspberry-pi-and-rpi-gpio-part-3)
- [wikipedia: Infrared](https://en.wikipedia.org/wiki/Infrared)
- [Wikipedia IMU](https://en.wikipedia.org/wiki/Inertial_measurement_unit)
- [pycreate2](https://pypi.python.org/pypi/pycreate2/0.7.3)
- [Kalman Filter](https://en.wikipedia.org/wiki/Kalman_filter)

## Setup

In [1]:
%matplotlib inline

from __future__ import division, print_function
import matplotlib.pyplot as plt
from math import pi
from IPython.display import HTML, display

# iRobot Create Sensors

<img src="pics/create.png" width="300px">

For this class we are using an iRobot Create 2 as our robot base. This robot is equiped with several sensors, but the ones we will use the most are:

- IR Sensors
    - Cliff detectors that detect the floor or the lack of one in the case of stairs
    - Bump detectors (really they are proximity sensors)
- Bump detectors which are switches that detect physical impact on the front
- Encoders which return the amount of wheel rotation for the left and right wheels
- Internal sensors for voltage and current
    - NiMH battery pack with 19 Ahr capacity. You can actually read this and determine how drained the Roomba is.

## Infrared Proximity

Infrared sensor are a very common type of proximity sensor. They basically send out a beam of modulated infrared light and looks for the returned signal. Sharp makes a wide range of IR sensors for various commercial applications with detection distances ranging from 2 cm to 6 m.

<img src="pics/ir_sensors.jpg" width="300px">

These sensor return a voltage that corresponds to a measured distance. The typical
sensor curve is shown below.

<img src="pics/ir_range_curve.png" width="400px">

- IR Issues
   - IR sensors are suseptable to being washed out by sunlight
   - Multipath: IR light from one sensor could be seen by another if the sensors are misaligned or the light bounces off a reflective surface
   - IR works best against surfaces that are orthogonal to the sensor, bright surfaces, and reflect IR. *Note:* cardboard is IR transparent, therefore you will not get a reflection off standard brown cardboard box material.

## Quaditure Encoders

Unfortunately, the Create doesn't have really good encoders like what 
is discussed here, but this will give you an idea of how they work.

<img src="pics/usdigital_encoder.jpg" width="300px">

There are many types of encoders, above is a US Digital encoder designed to
be mounted on a motor shaft.

<img src="pics/quadrature_encoder.gif" width="600px">

The optical encoder, shines a series of lights through an encoder disk and
the light is detected or not detected on the other side of the encoder disk
by some photoreceptors. A quadrature encoder has 2 signals, A and B, which
are phased such that they are *never* high or low at the same time. Depending
on the phase of the signals, the direction can be determined.

<img src="pics/quadrature_animation.gif" width="300px">

The animation shows the signals produced from the movement of the motor
shaft, with the encoder disk attached to it.

<img src="pics/quadrature_waveform.gif" width="300px">

Again, the wave form from A and B tells us if the wheel (motor shaft
and disk) are moving in the forward or reverse direction. Note that
forward/reverse are arbitrary and the engineer needs to determine
if CW or CCW is forward or reverse depending on how the sensor was
mounted to the robot.

<img src="pics/quadrature_resolution.gif" width="300px">

The resolution of the encoder is determine by how the 2 signals are
read.

- reading A and B on rising edge of A gives you the resolution of how many
  stripes there are on the disk
- reading on the rising and falling edge of A gives you twice the resolution
  of the number of stripes on the disk
- reading both A and B for both rising and falling edges gives you 4 times
  the resolution as the number of stripes on the disk

Now, obviously, the last option gives you the greatest resolution and the
best performance ... so why wouldn't you do it? If the speed of your
microcontroller is too slow and/or the speed of your wheel is too fast, you
could get stuck answering interrupts all the time and never doing anything
else. You have to balance your system constraints properly.

### Python Pseudo Code

```python

import time
from serial import Serial

count = 0
COUNTS_TO_METERS = 0.001  # this depends on the encoder system

def main_loop():
ser = Serial('/dev/tty.usbserial0', 115200)

while True:
  time.sleep(1)  # time depends on speed of robot
  position += count * COUNTS_TO_METERS
  count = 0

  # a super simple serial response to report position
  if ser.read() == 'p':
    ser.write(position)

# an interrupt that gets called every time A or B changes
# you can do this with RPi.GPIO on the raspberry pi
def interrupt_AB():
    A, B = readEncoderPins()
    if A ^ B == 1:
      count += 1
    else:
      count -= 1
```

- Encoder Issues:
    - Wheel slip can cause the encoder to read more distance travelled than what the robot has actually travelled. Hence, the Mars rovers do not rely on wheel encoders to determine how far they have travelled across the Marsian sand dunes.

# Additional Sensors: IMU

<img src="pics/imu-iso.jpg" width="300px">

Our Creates also have an inertial measurement unit (IMU) attached to the i2c bus of the Raspberry Pi.

blah ...

# Reading Create Sensors

To work the iRobot Create 2, we will use a library called [pycreate2](https://pypi.python.org/pypi/pycreate2/0.7.3) which was developed for this class. I suggest you take a look at the examples to help you write your code.

Typically when you build things (i.e., robots, airplanes, satellites, etc) you rarely get to test the entire thing. Instead, if you are building a targeting system for an aircraft, you work with pre-recorded data (from a radar that was flying in another aircraft) and ensure your targeting system interacts with that "canned" data properly.

We will do the same here. You will play with some pre-recorded data from the roomba and try to understand what is going on. This will help you when you get to work with the real roomba. 

## Bagit

`Bagit` stores data in a json file

In [ ]:
# let's create an instance of Bagit grab the pre-recorded data
bag = Bagit()
